In [1]:
# Base Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [2]:
# /content/drive/My Drive/fer2013.csv
def softmax(A):
    expA = np.exp(A)
    return expA / expA.sum(axis=1, keepdims=True)


def cost(Y, Yp):
    return -(Y*np.log(Yp)).sum()


def errorRate(Y, Yp):
    return np.mean(Y != Yp)


def oneZeroEncoding(y):
    N = len(y)
    K = len(set(y))
    
    index = np.zeros(shape=(N, K), dtype=int)
    
    for i, value in enumerate(y):
        index[i, value] = 1
        
    return index


def getData(balance_ones=True, Ntest=1000):
    # images are 48x48 = 2304 size vectors
    Y = []
    X = []
    first = True
    for line in open('/home/mrz/MyDrive/Education/Deep-Learning/Codes/LogisticRegression/facial-expression-recognition/fer2013/fer2013.csv'):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X) / 255.0, np.array(Y)

    # shuffle and split
    X, Y = shuffle(X, Y)
    Xtrain, Ytrain = X[:-Ntest], Y[:-Ntest]
    Xvalid, Yvalid = X[-Ntest:], Y[-Ntest:]

    if balance_ones:
        # balance the 1 class
        X0, Y0 = Xtrain[Ytrain!=1, :], Ytrain[Ytrain!=1]
        X1 = Xtrain[Ytrain==1, :]
        X1 = np.repeat(X1, 9, axis=0)
        Xtrain = np.vstack([X0, X1])
        Ytrain = np.concatenate((Y0, [1]*len(X1)))

    return Xtrain, Ytrain, Xvalid, Yvalid


In [3]:
Xtrain, Ytrain, Xvalid, Yvalid = getData()

In [4]:
Ytrain.shape

(39135,)

In [5]:
Xvalid.shape

(1000, 2304)

In [6]:
d = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0,}

for c in Ytrain:
    d[c] += 1
    
print(d)

{0: 4800, 1: 4779, 2: 4977, 3: 8760, 4: 5894, 5: 3892, 6: 6033}


In [7]:
oneZeroEncoding(Ytrain).shape

(39135, 7)

In [20]:
class ANN:
    def __init__(self, M):
        self.M = M
        
    def fit(self, X, Y, Xtest, Ytest, lr=1e-3, epochs=1000):
        N, D = X.shape
        K = len(set(Y))
        
        Y = oneZeroEncoding(Y)
        # Ytest = oneZeroEncoding(Ytest)
        
        self.W1 = np.random.randn(D, self.M) / np.sqrt(D)
        self.b1 = np.zeros(self.M)
        
        self.W2 = np.random.randn(self.M, K) / np.sqrt(self.M)
        self.b2 = np.zeros(K)
        
    
        
        costs = []
        for i in range(epochs):
            pY, Z = self.forward(X)
            
            self.W1 -= lr * ( X.T @ (pY - Y) )
            self.b1 -= lr * (pY-Y).sum()
            
            #if i%10 == 0:
            #    pYtest = self.forward(Xtest)
            #    c = cost(Ytest, pYtest)
            #    e = errorRate(np.argmax(Ytest, axis=1), np.argmax(pYtest, axis=1))
            #    
            #    print('Cost: {}, Error: {}, Epoch {}'.format(c, e, i))
            #    
            #    costs.append(c)
                
        ###
        #plt.plot(costs)
        #plt.show()
    
    def forward(self, X):
        Z = np.tanh( (X @ self.W1) + self.b1 )
        return softmax( (Z@self.W2) + self.b2 ), Z
    
#     def predict(self, X):
#         Yp = self.forward(X)
#         return np.argmax(Yp, axis=1)
    
#     def accuracy(self, X, Y):
#         prediction = self.predict(X)
#         return 1-errorRate(Y, prediction)
        

In [21]:
model = ANN(200)

In [22]:
model.fit(Xtrain, Ytrain, Xvalid, Yvalid, lr=1e-3, epochs=10)

(39135, 7)
(39135, 200)


In [99]:
model.predict(Xtest)

array([6, 2, 0, 1, 6, 6, 6, 6, 5, 6, 2, 4, 5, 0, 4, 2, 5, 0, 2, 0, 2, 4,
       1, 0, 3, 3, 1, 2, 3, 2, 1, 4, 3, 4, 6, 3, 2, 3, 2, 6, 2, 1, 5, 5,
       1, 4, 2, 3, 0, 3, 2, 4, 6, 4, 4, 0, 5, 3, 0, 2, 3, 2, 3, 3, 0, 0,
       3, 0, 4, 2, 1, 0, 1, 6, 4, 3, 2, 6, 3, 5, 4, 2, 5, 5, 2, 0, 0, 0,
       4, 3, 2, 6, 5, 3, 0, 4, 3, 3, 1, 4, 4, 3, 1, 1, 3, 3, 1, 1, 5, 3,
       2, 6, 3, 0, 6, 2, 5, 0, 3, 2, 3, 3, 1, 6, 2, 5, 5, 4, 2, 6, 2, 4,
       3, 4, 5, 2, 3, 6, 3, 0, 2, 3, 1, 3, 5, 5, 6, 5, 3, 3, 2, 6, 4, 1,
       0, 5, 5, 0, 4, 2, 3, 1, 3, 2, 3, 0, 0, 3, 1, 3, 6, 0, 3, 5, 4, 1,
       1, 5, 5, 4, 5, 0, 0, 0, 3, 5, 6, 4, 3, 5, 4, 6, 5, 0, 4, 3, 0, 6,
       3, 3, 1, 0, 3, 4, 2, 5, 6, 1, 0, 0, 0, 6, 4, 3, 3, 6, 1, 3, 1, 4,
       6, 0, 6, 3, 6, 5, 6, 2, 4, 0, 2, 2, 4, 6, 5, 0, 0, 0, 3, 3, 2, 2,
       0, 6, 0, 3, 4, 2, 3, 3, 3, 3, 3, 6, 3, 4, 2, 2, 5, 3, 1, 3, 0, 1,
       2, 1, 4, 5, 4, 3, 3, 0, 3, 1, 4, 5, 6, 5, 2, 6, 1, 3, 3, 4, 4, 1,
       1, 3, 0, 4, 3, 3, 4, 5, 3, 0, 0, 3, 3, 4, 0,

In [100]:
model.accuracy(Xtest, Ytest)

0.16400000000000003